In [0]:
print("🎯 DOWNLOADING BRAZILIAN E-COMMERCE DATASET")
print("="*70)

🎯 DOWNLOADING BRAZILIAN E-COMMERCE DATASET


### STEP 1: INSTALL KAGGLE & SET CREDENTIALS

In [0]:
print("\n1️⃣ Setting up Kaggle API...")

!pip install kaggle -q

import os

os.environ["KAGGLE_USERNAME"] = "YOUR_KAGGLE_USERNAME_HERE"
os.environ["KAGGLE_KEY"] = "YOUR_KAGGLE_KEY_HERE"

print("✅ Kaggle credentials configured!")


1️⃣ Setting up Kaggle API...
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
✅ Kaggle credentials configured!


### STEP 2: CREATE VOLUME FOR DATA

In [0]:
print("\n2️⃣ Creating storage volume...")

spark.sql("CREATE CATALOG IF NOT EXISTS brazil_project")
spark.sql("USE CATALOG brazil_project")

spark.sql("CREATE SCHEMA IF NOT EXISTS data")
spark.sql("CREATE VOLUME IF NOT EXISTS data.ecommerce_data")

print("✅ Volume created: brazil_project.data.ecommerce_data")


2️⃣ Creating storage volume...
✅ Volume created: brazil_project.data.ecommerce_data


## STEP 3: DOWNLOAD DATASET

In [0]:
print("\n3️⃣ Downloading Brazilian E-Commerce dataset...")

# Change to volume directory
%cd /Volumes/brazil_project/data/ecommerce_data

# Download dataset (Brazilian E-Commerce)
!kaggle datasets download -d olistbr/brazilian-ecommerce

print("✅ Download complete!")


3️⃣ Downloading Brazilian E-Commerce dataset...
/Volumes/brazil_project/data/ecommerce_data


/databricks/python/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


Dataset URL: https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce
License(s): CC-BY-NC-SA-4.0
 70%|██████████████████████████▋           | 30.0M/42.6M [00:00<00:00, 78.8MB/s]
100%|██████████████████████████████████████| 42.6M/42.6M [00:00<00:00, 55.4MB/s]
✅ Download complete!


## STEP 4: EXTRACT ZIP

In [0]:
%sh
cd /Volumes/brazil_project/data/ecommerce_data
unzip -o brazilian-ecommerce.zip
rm -f brazilian-ecommerce.zip
echo "Extracted files:"
ls -lh

Archive:  brazilian-ecommerce.zip
  inflating: olist_customers_dataset.csv  
  inflating: olist_geolocation_dataset.csv  
  inflating: olist_order_items_dataset.csv  
  inflating: olist_order_payments_dataset.csv  
  inflating: olist_order_reviews_dataset.csv  
  inflating: olist_orders_dataset.csv  
  inflating: olist_products_dataset.csv  
  inflating: olist_sellers_dataset.csv  
  inflating: product_category_name_translation.csv  
Extracted files:
total 121M
-rwxrwxrwx 1 spark-55896e9d-aee0-4f00-b3ae-b7 nogroup 8.7M Jan 25 12:47 olist_customers_dataset.csv
-rwxrwxrwx 1 spark-55896e9d-aee0-4f00-b3ae-b7 nogroup  59M Jan 25 12:47 olist_geolocation_dataset.csv
-rwxrwxrwx 1 spark-55896e9d-aee0-4f00-b3ae-b7 nogroup  15M Jan 25 12:47 olist_order_items_dataset.csv
-rwxrwxrwx 1 spark-55896e9d-aee0-4f00-b3ae-b7 nogroup 5.6M Jan 25 12:47 olist_order_payments_dataset.csv
-rwxrwxrwx 1 spark-55896e9d-aee0-4f00-b3ae-b7 nogroup  14M Jan 25 12:47 olist_order_reviews_dataset.csv
-rwxrwxrwx 1 spark-55

### LOAD ALL CSV FILES TO BRONZE

In [0]:
print("📥 Loading data to Bronze layer...")

# Create bronze schema if not exists
spark.sql("CREATE SCHEMA IF NOT EXISTS bronze")

# List of all CSV files
csv_files = [
    "olist_customers_dataset.csv",
    "olist_geolocation_dataset.csv", 
    "olist_order_items_dataset.csv",
    "olist_order_payments_dataset.csv",
    "olist_order_reviews_dataset.csv",
    "olist_orders_dataset.csv",
    "olist_products_dataset.csv",
    "olist_sellers_dataset.csv"
]

from pyspark.sql.functions import current_timestamp, lit

for csv_file in csv_files:
    table_name = csv_file.replace("olist_", "").replace("_dataset.csv", "")
    
    print(f"\n📋 Processing: {csv_file} -> bronze.{table_name}")
    
    try:
        # Full path to CSV
        csv_path = f"/Volumes/brazil_project/data/ecommerce_data/{csv_file}"
        
        # Read CSV
        df = spark.read.option("header", True).option("inferSchema", True).csv(csv_path)
        
        # Add metadata columns
        df = df.withColumn("_ingestion_time", current_timestamp()) \
               .withColumn("_source_file", lit(csv_file))
        
        # Write to Delta table
        df.write.mode("overwrite").saveAsTable(f"brazil_project.bronze.{table_name}")
        
        row_count = df.count()
        print(f"   ✅ Success: {row_count:,} rows loaded")
        
    except Exception as e:
        print(f"   ❌ Error loading {csv_file}: {str(e)[:100]}...")

print("\n" + "="*70)
print("📊 LOADING SUMMARY")
print("="*70)

# Show all tables created
tables = spark.sql("SHOW TABLES IN bronze").collect()
for table in tables:
    table_name = table.tableName
    count = spark.sql(f"SELECT COUNT(*) as cnt FROM bronze.{table_name}").collect()[0]['cnt']
    print(f"📦 {table_name:20} : {count:>10,} rows")

📥 Loading data to Bronze layer...

📋 Processing: olist_customers_dataset.csv -> bronze.customers
   ✅ Success: 99,441 rows loaded

📋 Processing: olist_geolocation_dataset.csv -> bronze.geolocation
   ✅ Success: 1,000,163 rows loaded

📋 Processing: olist_order_items_dataset.csv -> bronze.order_items
   ✅ Success: 112,650 rows loaded

📋 Processing: olist_order_payments_dataset.csv -> bronze.order_payments
   ✅ Success: 103,886 rows loaded

📋 Processing: olist_order_reviews_dataset.csv -> bronze.order_reviews
   ✅ Success: 104,162 rows loaded

📋 Processing: olist_orders_dataset.csv -> bronze.orders
   ✅ Success: 99,441 rows loaded

📋 Processing: olist_products_dataset.csv -> bronze.products
   ✅ Success: 32,951 rows loaded

📋 Processing: olist_sellers_dataset.csv -> bronze.sellers
   ✅ Success: 3,095 rows loaded

📊 LOADING SUMMARY
📦 customers            :     99,441 rows
📦 geolocation          :  1,000,163 rows
📦 order_items          :    112,650 rows
📦 order_payments       :    103,886 r

## VERIFY & SHOW SAMPLE DATA

In [0]:
print("🔍 VERIFYING DATA LOAD")
print("="*70)

# Show schema of key tables
print("\n📋 TABLE SCHEMAS:")

key_tables = ["customers", "orders", "order_items", "products"]
for table in key_tables:
    print(f"\n{table.upper()}:")
    spark.table(f"brazil_project.bronze.{table}").printSchema()

# Show sample data
print("\n" + "="*70)
print("📊 SAMPLE DATA")
print("="*70)

print("\n👥 CUSTOMERS (first 3 rows):")
spark.sql("SELECT * FROM bronze.customers LIMIT 3").show(truncate=False)

print("\n🛒 ORDERS (first 3 rows):")
spark.sql("SELECT * FROM bronze.orders LIMIT 3").show(truncate=False)

print("\n📦 ORDER ITEMS (first 3 rows):")
spark.sql("SELECT * FROM bronze.order_items LIMIT 3").show(truncate=False)

print("\n💰 ORDER PAYMENTS (first 3 rows):")
spark.sql("SELECT * FROM bronze.order_payments LIMIT 3").show(truncate=False)

# Data quality check
print("\n" + "="*70)
print("✅ DATA QUALITY CHECK")
print("="*70)

print("\n🔍 Checking for NULL values in key columns:")

checks = [
    ("customers", "customer_id"),
    ("orders", "order_id"),
    ("orders", "customer_id"),
    ("order_items", "order_id"),
    ("products", "product_id")
]

for table, column in checks:
    try:
        null_count = spark.sql(f"""
            SELECT COUNT(*) as null_count 
            FROM bronze.{table} 
            WHERE {column} IS NULL
        """).collect()[0]['null_count']
        
        total_count = spark.sql(f"SELECT COUNT(*) as total FROM bronze.{table}").collect()[0]['total']
        
        if null_count == 0:
            print(f"   ✅ {table}.{column}: No NULLs ({total_count:,} rows)")
        else:
            print(f"   ⚠️  {table}.{column}: {null_count:,} NULLs out of {total_count:,}")
    except:
        print(f"   ❌ Could not check {table}.{column}")


🔍 VERIFYING DATA LOAD

📋 TABLE SCHEMAS:

CUSTOMERS:
root
 |-- customer_id: string (nullable = true)
 |-- customer_unique_id: string (nullable = true)
 |-- customer_zip_code_prefix: integer (nullable = true)
 |-- customer_city: string (nullable = true)
 |-- customer_state: string (nullable = true)
 |-- _ingestion_time: timestamp (nullable = true)
 |-- _source_file: string (nullable = true)


ORDERS:
root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: timestamp (nullable = true)
 |-- order_approved_at: timestamp (nullable = true)
 |-- order_delivered_carrier_date: timestamp (nullable = true)
 |-- order_delivered_customer_date: timestamp (nullable = true)
 |-- order_estimated_delivery_date: timestamp (nullable = true)
 |-- _ingestion_time: timestamp (nullable = true)
 |-- _source_file: string (nullable = true)


ORDER_ITEMS:
root
 |-- order_id: string (nullable = true)
 |-- order